In [1]:
import sys
print(sys.executable)

/home/tomislav/mambaforge-pypy3/envs/houses_env/bin/python3


In [49]:
import pandas as pd
import os


dataset_path = "../data/house-prices-advanced-regression-techniques/"
tr_df = pd.read_csv(os.path.join(dataset_path, "train.csv"))
test_df = pd.read_csv(os.path.join(dataset_path, "test.csv"))

In [26]:
print(tr_df.shape)
print(test_df.shape)

(1460, 81)
(1459, 80)


In [31]:
pd.set_option("display.max_columns", None)
df = tr_df.query('MasVnrType.isnull() & MasVnrArea.isnull()')

#df["MasVnrArea"] = 0.0

In [29]:
tr_df.loc[tr_df["MasVnrArea"].isnull() & tr_df["MasVnrType"].isnull(), "MasVnrArea"] = 0.0

In [36]:
pd.set_option("display.max_rows", None)
tr_df["MasVnrArea"].isnull().sum()

0

In [51]:
tr_df.dropna(subset=["Electrical"], inplace=True)

In [52]:
tr_df["Electrical"].isnull().sum()

0

In [62]:
nullseries = tr_df.isnull().sum()
print(nullseries[nullseries > 0])

LotFrontage      259
Alley           1368
MasVnrType       871
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
FireplaceQu      689
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1452
Fence           1178
MiscFeature     1405
dtype: int64


In [7]:
import sklearn